# Model Training

### SVM model 

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2



In [32]:
df = pd.read_csv("/Users/khushpatel/Desktop/ML Projects/Emotion Detection/audio_features.csv")


In [33]:
df['label'].unique()

array(['YAF_disgust', 'OAF_Pleasant_surprise', 'OAF_happy', 'YAF_sad',
       'YAF_happy', 'YAF_neutral', 'OAF_Fear', 'OAF_angry',
       'YAF_pleasant_surprised', 'YAF_fear', 'OAF_neutral', 'OAF_disgust',
       'YAF_angry', 'OAF_Sad'], dtype=object)

In [34]:
#Merging emotions of YAF and OAF 

def normalized_labels(label):
    return label.split('_')[1].lower()

df['label'] = df['label'].apply(normalized_labels)
print(df['label'].unique())


['disgust' 'pleasant' 'happy' 'sad' 'neutral' 'fear' 'angry']


In [35]:
#OneHot encoding 

encoder = OneHotEncoder()
encoder.fit(df[['label']])
encoded_labels = pd.DataFrame(encoder.transform(df[['label']]).toarray(), columns = encoder.get_feature_names_out(['label']))
print(encoded_labels)
df_encoded = pd.concat([df, encoded_labels], axis=1)

# Drop the original 'label' column
df_encoded.drop(columns=['label'], inplace=True)

df_encoded.columns

      label_angry  label_disgust  label_fear  label_happy  label_neutral  \
0             0.0            1.0         0.0          0.0            0.0   
1             0.0            1.0         0.0          0.0            0.0   
2             0.0            1.0         0.0          0.0            0.0   
3             0.0            1.0         0.0          0.0            0.0   
4             0.0            1.0         0.0          0.0            0.0   
...           ...            ...         ...          ...            ...   
2795          0.0            0.0         0.0          0.0            0.0   
2796          0.0            0.0         0.0          0.0            0.0   
2797          0.0            0.0         0.0          0.0            0.0   
2798          0.0            0.0         0.0          0.0            0.0   
2799          0.0            0.0         0.0          0.0            0.0   

      label_pleasant  label_sad  
0                0.0        0.0  
1                0.

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '163', '164', '165', 'label_angry', 'label_disgust', 'label_fear',
       'label_happy', 'label_neutral', 'label_pleasant', 'label_sad'],
      dtype='object', length=173)

In [36]:
target = ['label_angry', 'label_disgust', 'label_fear',
       'label_happy', 'label_neutral', 'label_pleasant', 'label_sad']

X = df_encoded.drop(columns=target)
Y = df_encoded[target]

scaled = StandardScaler()
X_scaled = scaled.fi

X_train,X_test,y_train,y_test =  train_test_split(X,Y,test_size=0.2,random_state=42)

In [46]:
model = Sequential()
model.add(Dense(32,input_shape=(X_train.shape[1],),activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(16,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(8,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dense(Y.shape[1],activation='softmax'))

optimier = Adam(learning_rate=0.0001)

model.compile(optimizer=optimier,loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=100,batch_size=1,validation_split=0.1)

loss ,accuracy = model.evaluate(X_test,y_test)
print(f'Test Accuracy {accuracy *100:.2f}%')

Epoch 1/100
2016/2016 [==============================] - 2s 648us/step - loss: 5.9183 - accuracy: 0.1954 - val_loss: 2.1771 - val_accuracy: 0.3304
Epoch 2/100
2016/2016 [==============================] - 1s 557us/step - loss: 1.8855 - accuracy: 0.3611 - val_loss: 1.8229 - val_accuracy: 0.4420
Epoch 3/100
2016/2016 [==============================] - 1s 613us/step - loss: 1.5208 - accuracy: 0.5293 - val_loss: 1.4009 - val_accuracy: 0.6071
Epoch 4/100
2016/2016 [==============================] - 1s 566us/step - loss: 1.2433 - accuracy: 0.6587 - val_loss: 1.1527 - val_accuracy: 0.7054
Epoch 5/100
2016/2016 [==============================] - 1s 566us/step - loss: 1.0710 - accuracy: 0.7178 - val_loss: 1.0511 - val_accuracy: 0.7411
Epoch 6/100
2016/2016 [==============================] - 1s 557us/step - loss: 0.9405 - accuracy: 0.7738 - val_loss: 0.9228 - val_accuracy: 0.7589
Epoch 7/100
2016/2016 [==============================] - 1s 556us/step - loss: 0.8499 - accuracy: 0.8070 - val_loss: 0

In [47]:
model.save("model.h5")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [49]:
from tensorflow.keras.models import load_model

test_data = pd.read_csv('test.csv')

feature_columns = [f'{i}' for i in range(166)]
X_new = test_data[feature_columns].values

model = load_model('model.h5')

predictions = model.predict(X_new)

# Convert probabilities to class labels
predicted_labels = predictions.argmax(axis=1)
emotion_labels = ['angry', 'sad', 'disgust', 'happy', 'pleasant_surprised', 'neutral', 'fear']

predicted_class_names = [emotion_labels[i] for i in predicted_labels]

print(f'class:{predicted_class_names}')

1/1 [==============================] - 0s 43ms/step
class:['sad']
